# Efficient Yelp API Calls (Core)


For this assignment, you will be working with the Yelp API.

As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

Save your notebook, commit the change to your repository and submit the repository URL for this assignment.


---

## Imports 

In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [67]:
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [68]:
!pip install yelpapi
!pip install tqdm

### Credientials & Accessing the API

In [69]:
with open('/Users/valma/.secret/yelp_api.json','r') as f: 
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [70]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

### Define Search

In [71]:
location = 'San Antonio, Texas 78209'
term = 'mexican'

In [72]:
location.split(',')[0]

'San Antonio'

In [73]:
folder = 'Data/'

os.makedirs(folder, exist_ok=True)

JSON_FILE = folder+f"{location.split(',')[0]}-{term}.json"

In [74]:
JSON_FILE

'Data/San Antonio-mexican.json'

### Create JSON file

In [75]:
file_exists = os.path.isfile(JSON_FILE)
if file_exists == False:    
    folder = os.path.dirname(JSON_FILE)
    
    if len(folder)>0:
        os.makedirs(folder, exist_ok = True)
        
        
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    with open(JSON_FILE, 'w') as f:
          json.dump([], f)
        
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/San Antonio-mexican.json already exists.


In [76]:
os.path.isfile(JSON_FILE)

True

### Figure out how many pages of results we will need

In [77]:
results = yelp_api.search_query(term = term, location = location)

In [78]:
type(results)

dict

In [79]:
len(results)

3

In [80]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [81]:
results['total']

895

In [82]:
results['region']

{'center': {'longitude': -98.45603942871094, 'latitude': 29.48562181547507}}

In [83]:
results['businesses']

[{'id': '1PQ-elmPdTvWDQ4JnFaupA',
  'alias': 'paloma-blanca-mexican-cuisine-san-antonio-2',
  'name': 'Paloma Blanca Mexican Cuisine',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/jQga3Nlc-4356t-DQKf_SA/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/paloma-blanca-mexican-cuisine-san-antonio-2?adjust_creative=IMMkKxNodFLQzqkEcp4aHA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=IMMkKxNodFLQzqkEcp4aHA',
  'review_count': 1081,
  'categories': [{'alias': 'mexican', 'title': 'Mexican'},
   {'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'},
   {'alias': 'catering', 'title': 'Caterers'}],
  'rating': 4.0,
  'coordinates': {'latitude': 29.47960940684702,
   'longitude': -98.46351734690846},
  'transactions': ['delivery'],
  'price': '$$',
  'location': {'address1': '5800 Broadway St',
   'address2': 'Ste 300',
   'address3': '',
   'city': 'San Antonio',
   'zip_code': '78209',
   'country': 'US',
   'state': 'TX',
   'display_add

In [84]:
pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,1PQ-elmPdTvWDQ4JnFaupA,paloma-blanca-mexican-cuisine-san-antonio-2,Paloma Blanca Mexican Cuisine,https://s3-media2.fl.yelpcdn.com/bphoto/jQga3N...,False,https://www.yelp.com/biz/paloma-blanca-mexican...,1081,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 29.47960940684702, 'longitude': -...",[delivery],$$,"{'address1': '5800 Broadway St', 'address2': '...",+12108226151,(210) 822-6151,985.335937
1,zulD0Ns_524wMuCAFlXxRA,tlahco-mexican-kitchen-san-antonio,Tlahco Mexican kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/PAEto8...,False,https://www.yelp.com/biz/tlahco-mexican-kitche...,784,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 29.5006724654481, 'longitude': -9...","[pickup, delivery]",$$,"{'address1': '6702 San Pedro Ave', 'address2':...",+12104550135,(210) 455-0135,4475.940544
2,MccLbDYazJ2iJutGsbXT5A,santa-diabla-san-antonio-3,Santa Diabla,https://s3-media3.fl.yelpcdn.com/bphoto/9_3Xju...,False,https://www.yelp.com/biz/santa-diabla-san-anto...,90,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 29.44077, 'longitude': -98.48555}",[],$$,"{'address1': '906 E Elmira St', 'address2': ''...",+12103147951,(210) 314-7951,5745.299240
3,CLbzZGsLKcUf4G269KEu_Q,rosarios-comidamex-and-bar-san-antonio-2,Rosario's ComidaMex & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TXifgw...,False,https://www.yelp.com/biz/rosarios-comidamex-an...,3597,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 29.41725, 'longitude': -98.48987}","[pickup, delivery]",$$,"{'address1': '722 S Saint Mary's St', 'address...",+12102231806,(210) 223-1806,8274.885928
4,1VDoVBHq1cFiW66diTMv5w,soluna-cocina-mexicana-san-antonio,Soluna Cocina Mexicana,https://s3-media1.fl.yelpcdn.com/bphoto/hMWUya...,False,https://www.yelp.com/biz/soluna-cocina-mexican...,545,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 29.5066405659216, 'longitude': -9...","[pickup, delivery]",$$,"{'address1': '7959 Broadway St', 'address2': '...",+12109308070,(210) 930-8070,2553.790300
5,W-KGHdbsz_jKmEuyF-Xukw,la-fogata-san-antonio-6,La Fogata,https://s3-media2.fl.yelpcdn.com/bphoto/v0v9bw...,False,https://www.yelp.com/biz/la-fogata-san-antonio...,39,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.5,"{'latitude': 29.4875, 'longitude': -98.44791}",[],NaN,"{'address1': '1133 Austin Hwy', 'address2': ''...",+12108248686,(210) 824-8686,812.578526
6,T1OGlUfG3rU6VrDHMBSJ5A,taqueria-cazadores-san-antonio-2,Taqueria Cazadores,https://s3-media1.fl.yelpcdn.com/bphoto/-F7Irg...,False,https://www.yelp.com/biz/taqueria-cazadores-sa...,188,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 29.48494, 'longitude': -98.43101}","[pickup, delivery]",$$,"{'address1': '927 Rittiman Rd', 'address2': ''...",+12108240175,(210) 824-0175,2427.305858
7,wf6rez0-mpDypXjC_ko90w,el-catrin-restaurant-and-bar-san-antonio,El Catrin Restaurant & Bar,https://s3-media3.fl.yelpcdn.com/bphoto/gjGMco...,False,https://www.yelp.com/biz/el-catrin-restaurant-...,26,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 29.49079088125391, 'longitude': -...",[],NaN,"{'address1': '21 Brees Blvd', 'address2': '', ...",+12105001347,(210) 500-1347,624.199616
8,PAhIvZh810jjYai3BPzshQ,taqueria-el-charro-san-antonio-3,Taqueria El Charro,https://s3-media1.fl.yelpcdn.com/bphoto/Q5Ykry...,False,https://www.yelp.com/biz/taqueria-el-charro-sa...,79,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 29.483580519870884, 'longitude': ...",[],$,"{'address1': '2351 Harry Wurzbach Rd', 'addres...",+12102579410,(210) 257-9410,2419.452279
9,T8WC3EmJhNeliIjLXwQ8qw,la-fonda-on-main-san-antonio,La Fonda on Main,https://s3-media3.fl.yelpcdn.com/bphoto/5PcAdz...,False,https://www.yelp.com/biz/la-fonda-on-main-san-...,966,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 29.

In [85]:
results_per_page = len(results['businesses'])
results_per_page

20

In [86]:
(results['total'])/ results_per_page


44.75

In [87]:
n_pages = math.ceil((results['total'])/ results_per_page)
n_pages

45

### Set up a progress bar in our for loop.

In [88]:
for i in tqdm_notebook(range(1,n_pages+1)):
    try:
        
        time.sleep(.2)
    
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
        
        n_results = len(previous_results)
        
        results = yelp_api.search_query(location = location, term = term,
                                   offset = n_results+1)

        previous_results.extend(results['businesses'])
        
        with open(JSON_FILE, 'w') as f:
            json.dump(previous_results, f)

            
    except Exception as e:
        print(' [!] ERROR', e)

  0%|          | 0/45 [00:00<?, ?it/s]

### Opening JSON File with Pandas

In [89]:
df = pd.read_json(JSON_FILE)

In [90]:
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zulD0Ns_524wMuCAFlXxRA,tlahco-mexican-kitchen-san-antonio,Tlahco Mexican kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/PAEto8...,False,https://www.yelp.com/biz/tlahco-mexican-kitche...,784,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 29.5006724654481, 'longitude': -9...","[pickup, delivery]",$$,"{'address1': '6702 San Pedro Ave', 'address2':...",+12104550135,(210) 455-0135,4475.940544
1,MccLbDYazJ2iJutGsbXT5A,santa-diabla-san-antonio-3,Santa Diabla,https://s3-media3.fl.yelpcdn.com/bphoto/9_3Xju...,False,https://www.yelp.com/biz/santa-diabla-san-anto...,90,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 29.44077, 'longitude': -98.48555}",[],$$,"{'address1': '906 E Elmira St', 'address2': ''...",+12103147951,(210) 314-7951,5745.299240
2,CLbzZGsLKcUf4G269KEu_Q,rosarios-comidamex-and-bar-san-antonio-2,Rosario's ComidaMex & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TXifgw...,False,https://www.yelp.com/biz/rosarios-comidamex-an...,3597,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 29.41725, 'longitude': -98.48987}","[pickup, delivery]",$$,"{'address1': '722 S Saint Mary's St', 'address...",+12102231806,(210) 223-1806,8274.885928
3,1VDoVBHq1cFiW66diTMv5w,soluna-cocina-mexicana-san-antonio,Soluna Cocina Mexicana,https://s3-media1.fl.yelpcdn.com/bphoto/hMWUya...,False,https://www.yelp.com/biz/soluna-cocina-mexican...,545,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 29.5066405659216, 'longitude': -9...","[pickup, delivery]",$$,"{'address1': '7959 Broadway St', 'address2': '...",+12109308070,(210) 930-8070,2553.790300
4,W-KGHdbsz_jKmEuyF-Xukw,la-fogata-san-antonio-6,La Fogata,https://s3-media2.fl.yelpcdn.com/bphoto/v0v9bw...,False,https://www.yelp.com/biz/la-fogata-san-antonio...,39,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.5,"{'latitude': 29.4875, 'longitude': -98.44791}",[],NaN,"{'address1': '1133 Austin Hwy', 'address2': ''...",+12108248686,(210) 824-8686,812.578526


In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 894 entries, 0 to 893
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             894 non-null    object 
 1   alias          894 non-null    object 
 2   name           894 non-null    object 
 3   image_url      894 non-null    object 
 4   is_closed      894 non-null    bool   
 5   url            894 non-null    object 
 6   review_count   894 non-null    int64  
 7   categories     894 non-null    object 
 8   rating         894 non-null    float64
 9   coordinates    894 non-null    object 
 10  transactions   894 non-null    object 
 11  price          623 non-null    object 
 12  location       894 non-null    object 
 13  phone          894 non-null    object 
 14  display_phone  894 non-null    object 
 15  distance       894 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 105.8+ KB


In [92]:
df.duplicated(subset='id').sum()

0

### File Compression - File Conversion

In [93]:
final_df = JSON_FILE.replace('.json','.csv.gz')
final_df

'Data/San Antonio-mexican.csv.gz'

In [94]:
df.to_csv(final_df,compression = 'gzip', index = False)

### File Size Comparison

In [95]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 869,778 Bytes
CSV.GZ FILE: 123,276 Bytes
the csv.gz is 7.055533923878127 times smaller!
